<a href="https://colab.research.google.com/github/daraadeniyi3116-create/ACCESS-6.0-SAP-Data-Science-Track/blob/main/Mini_Project_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np

from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("Mounted!")
else:
    print("Drive is already mounted.")

Drive is already mounted.


## Feature Engineering

In [13]:
df = pd.read_csv('/content/Warehouse_and_Retail_Sales_Cleaned.csv')


# --- 1. Date Feature: Seasonality ---
# Instead of just using "Month" (which is just a number), I grouped them into Seasons.
# This helps the model capture behaviors like "Holiday Shopping" vs "Summer Break".
def get_season(month):
    if month in [11, 12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['MONTH'].apply(get_season)

# --- 2. Aggregated Feature: Total Volume ---
# The dataset splits sales into Retail and Warehouse.
# To understand true demand, I combined them.
df['Total_Sales'] = df['RETAIL SALES'] + df['WAREHOUSE SALES']

# --- 3. Advanced Feature: Relative Performance ---
# This is a key feature. It calculates the average sales for each Item Type (e.g., Wine, Beer),
# then compares this specific row to that average.
# 1.0 = Average performance.
# >1.0 = Outperforming its category.
item_type_means = df.groupby('ITEM TYPE')['Total_Sales'].transform('mean')
df['Relative_Performance'] = df['Total_Sales'] / (item_type_means + 1) # +1 avoids division by zero

df[['MONTH', 'Season', 'ITEM TYPE', 'Total_Sales', 'Relative_Performance']].head()

,MONTH,Season,ITEM TYPE,Total_Sales,Relative_Performance
0,1,Winter,WINE,2.00,0.179463
1,1,Winter,WINE,4.00,0.358927
2,1,Winter,BEER,1.00,0.005937
3,1,Winter,WINE,1.00,0.089732
4,1,Winter,WINE,0.82,0.073580


## Encoding Decisions

In [14]:
df.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,TOTAL_SALES,RETAIL_RATIO,Season,Total_Sales,Relative_Performance
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0,2.00,0.000000,Winter,2.00,0.179463
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0,4.00,0.000000,Winter,4.00,0.358927
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0,1.00,0.000000,Winter,1.00,0.005937
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0,1.00,0.000000,Winter,1.00,0.089732
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0,0.82,0.998782,Winter,0.82,0.073580


I identified 4 columns containing text/non-numeric data. Here is the encoding strategy I will use for each:

* **Column:** "Season" & "ITEM TYPE"
    * **Decision: One-Hot Encoding**
    * **Reason:** These are **nominal categories** (they have no inherent order). "Spring" is not mathematically greater than "Winter," and "Wine" is not greater than "Beer." One-Hot Encoding creates a separate binary column for each category (e.g., "Season_Winter", "Item_Wine"), which allows the model to recognize them distinctively without assuming a false hierarchy.

* **Column:** "SUPPLIER"
    * **Decision: Label Encoding**
    * **Reason:** This column has **high cardinality** (hundreds of unique suppliers). Using One-Hot Encoding here would create hundreds of new columns (the "Curse of Dimensionality"), making the dataset massive and slow to train. Label Encoding is efficient because it assigns a unique integer to each supplier without changing the dataset's shape.

* **Column:** "ITEM DESCRIPTION"
    * **Decision: Label Encoding**
    * **Reason:** This column contains thousands of unique variations of text. One-Hot Encoding is impossible here due to the size. Label Encoding allows us to turn this text into numbers so we don't have to drop the column, allowing the model to potentially find patterns in specific item descriptions.

## Scaling Decisions

In [20]:
df.describe()

,YEAR,MONTH,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,TOTAL_SALES,RETAIL_RATIO,Total_Sales,Relative_Performance
count,307645.000000,307645.000000,307645.000000,307645.000000,307645.000000,307645.000000,307645.000000,307645.000000,307645.000000
mean,2018.438525,6.423862,7.024003,6.936465,25.294597,32.318600,0.429798,32.318600,0.927262
std,1.083061,3.461812,30.986095,30.237195,249.916798,266.486255,1.616856,266.486255,2.950824
min,2017.000000,1.000000,-6.490000,-38.490000,-7800.000000,-7800.000000,-104.210526,-7800.000000,-1.920301
25%,2017.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.078910
50%,2019.000000,7.000000,0.320000,0.000000,1.000000,2.680000,0.242332,2.680000,0.179463
75%,2019.000000,9.000000,3.260000,3.000000,5.000000,10.500000,0.994152,10.500000,0.629875
max,2020.000000,12.000000,2739.000000,1990.830000,18317.000000,19491.000000,830.000000,19491.000000,123.175603


**Columns to Scale**: RETAIL SALES, WAREHOUSE SALES, TOTAL_SALES, and Relative_Performance.

**Scaler Selected**: MinMaxScaler

**Reasoning**: I examined the df.describe() table and found a huge disparity in the ranges of my features:

- Categorical features (like the One-Hot encoded Seasons) are all 0 or 1.

- Date features (like Month) only range from 1 to 12.

- Sales features (like TOTAL_SALES), however, range from -7,800 up to 19,491.

If I feed this directly into a regression model, the model will treat the Sales columns as thousands of times more important than the Season or Month simply because the numbers are bigger. I will use MinMaxScaler to squeeze these large sales figures into a range of 0 to 1, ensuring they don't overpower the other features.

## Model Thinking

**Target Variable**: TOTAL_SALES

**Problem Type**: Regression.

**Reasoning**: I am trying to predict the exact volume of sales, which is a continuous numerical value (e.g., selling 50 units vs. 500 units). This is different from Classification, where I would be predicting a label like "High Performer" or "Low Performer." Since the output can be any number on a scale, and I have historical data to learn from.